In [68]:
import os
from configparser import ConfigParser
import pandas as pd

# 1. Configure SAP NetWeaver SDK

In [69]:
proj_dir = os.getcwd()

In [70]:
os.environ['SAPNWRFC_HOME'] = proj_dir + '\\nwrfcsdk' 

In [71]:
from pyrfc import Connection, ABAPRuntimeError

# 2. Locad configuration

In [72]:
configur = ConfigParser()

In [73]:
configur.read('config.ini')

['config.ini']

In [74]:
ASHOST=configur.get('SAP','host')
CLIENT=configur.get('SAP','client')
SYSNR=configur.get('SAP','instance')
USER=configur.get('SAP','username')
PASSWD=configur.get('SAP','password')

# 3. Connect to SAP

In [75]:
conn = Connection(
    ashost=ASHOST, 
    sysnr=SYSNR, 
    client=CLIENT, 
    user=USER, 
    passwd=PASSWD,
    lang="EN"
    )

In [76]:
def conver_results_to_object(result):
    results = []
    for row in result['DATA']:
        
            d = row['WA'].split('|')
            fields = result['FIELDS']
            o = {}
            for i,v in enumerate(d):
                o[fields[i]['FIELDNAME']] = v
            results.append(o)
    return results

# 4. Define Functions

## 4.1 Get PO Invoices

In [77]:
def get_po_invoices(conn,po):
    po_results = []
    kwargs = dict(
                QUERY_TABLE = 'EKBE',
                DELIMITER = '|',
                NO_DATA = '',
                FIELDS = [ 
                    {'FIELDNAME':'EBELN'} ,
                    {'FIELDNAME':'EBELP'} ,
                    {'FIELDNAME':'ZEKKN'} ,
                    {'FIELDNAME':'VGABE'} ,
                    {'FIELDNAME':'GJAHR'} ,
                    {'FIELDNAME':'BELNR'} ,
                    {'FIELDNAME':'BUZEI'} ,
                        ],
                ROWSKIPS = 0 ,
                ROWCOUNT = 500,
                OPTIONS = [
                    "MANDT = '"+ CLIENT +"'", 
                    " AND EBELN = '"+ po +"'",
                    " AND VGABE = '2'"
                    ]
            )
    try:
        result = conn.call('RFC_READ_TABLE',**kwargs)
        po_results = conver_results_to_object(result)
    except ABAPRuntimeError as error:
        print(error.message)
    
    return po_results

## 4.2 Get FI Document from PO Invoice

In [78]:
def get_fi_doc(conn,ObjectKey):
    po_results = []
    kwargs = dict(
                QUERY_TABLE = 'BKPF',
                DELIMITER = '|',
                NO_DATA = '',
                FIELDS = [ 
                    {'FIELDNAME':'BUKRS'} ,
                    {'FIELDNAME':'BELNR'} ,
                    {'FIELDNAME':'GJAHR'} ,
                    {'FIELDNAME':'BLART'} ,
                    {'FIELDNAME':'AWTYP'} ,
                    {'FIELDNAME':'AWKEY'} ,
                        ],
                ROWSKIPS = 0 ,
                ROWCOUNT = 500,
                OPTIONS = [
                    "MANDT = '"+ CLIENT +"'", 
                    " AND AWKEY = '"+ ObjectKey +"'",
                    ]
            )
    try:
        result = conn.call('RFC_READ_TABLE',**kwargs)
        po_results = conver_results_to_object(result)
    except ABAPRuntimeError as error:
        print(error.message)
    
    return po_results

## 4.3 Get FI Document items

In [79]:
def get_fi_doc_items(conn,BUKRS,BELNR,GJAHR):
    results = []
    kwargs = dict(
                QUERY_TABLE = 'BSEG',
                DELIMITER = '|',
                NO_DATA = '',
                FIELDS = [ 
                    {'FIELDNAME':'BUKRS'} ,
                    {'FIELDNAME':'BELNR'} ,
                    {'FIELDNAME':'GJAHR'} ,
                    {'FIELDNAME':'BUZEI'} ,
                    {'FIELDNAME':'AUGDT'} ,
                    {'FIELDNAME':'AUGCP'} ,
                    {'FIELDNAME':'AUGBL'} ,
                    {'FIELDNAME':'BSCHL'} ,
                        ],
                ROWSKIPS = 0 ,
                ROWCOUNT = 500,
                OPTIONS = [
                    "MANDT = '"+ CLIENT +"'", 
                    " AND BUKRS = '"+ BUKRS +"'",
                    " AND BELNR = '"+ BELNR +"'",
                    " AND GJAHR = '"+ GJAHR +"'",
                    " AND BSCHL = '31'",
                    ]
            )
    try:
        result = conn.call('RFC_READ_TABLE',**kwargs)
        results = conver_results_to_object(result)
    except ABAPRuntimeError as error:
        print(error.message)
    
    return results

# 5. Read PO Invoices

In [80]:
#po_data = get_po_invoices(conn,'4500019671')

In [81]:
po_list = [
    '0004056101',
    '0004055874',
    '0004055873',
    '0004057015'
]

In [82]:
po_data = []
for po in po_list:
    result = get_po_invoices(conn,po)
    po_data += result

In [83]:
df_po_ri = pd.DataFrame(data=po_data)

In [84]:
df_po_ri['AWKEY'] = df_po_ri['BELNR'] + df_po_ri['GJAHR']

In [85]:
AWKEY_docs = list(df_po_ri['AWKEY'].drop_duplicates())

# 6. Retriave FI documents 

In [86]:
fi_doc_hd = []
for objKey in AWKEY_docs:
    result = get_fi_doc(conn, objKey)
    fi_doc_hd += result

In [87]:
df_fi_doc_hd = pd.DataFrame(data=fi_doc_hd)

In [88]:
fi_doc_it = []
for fi_doc in fi_doc_hd:
    result = get_fi_doc_items(conn, fi_doc['BUKRS'],fi_doc['BELNR'],fi_doc['GJAHR'])
    fi_doc_it += result

In [89]:
df_fi_doc_it = pd.DataFrame(data=fi_doc_it)

In [90]:
df_fi_doc_it.loc[ df_fi_doc_it['AUGDT'] == '00000000' ,'AUGDT'] = ''

In [91]:
df_fi_doc_it.loc[ df_fi_doc_it['AUGCP'] == '00000000' ,'AUGCP'] = ''

In [92]:
df_fi_doc = pd.merge(df_fi_doc_hd,df_fi_doc_it,on=['BUKRS','BELNR','GJAHR'])

In [93]:
df_1 = pd.merge(df_po_ri,df_fi_doc,on='AWKEY')

In [94]:
df_1[['EBELN','GJAHR_x','BELNR_x','BELNR_y','GJAHR_y','AUGDT','AUGCP','AUGBL']]

,EBELN,GJAHR_x,BELNR_x,BELNR_y,GJAHR_y,AUGDT,AUGCP,AUGBL
0,0004056101,2024,0000133213,0000133213,2024,20241126,20241126,0000511158
1,0004056101,2024,0000134649,0000134649,2024,20250127,20250127,0000510110
2,0004056101,2025,0000135863,0000135863,2025,20250311,20250311,0000510287
3,0004056101,2025,0000136604,0000136604,2025,20250410,20250410,0000510362
4,0004056101,2025,0000137132,0000137132,2025,20250506,20250506,0000622011
5,0004056101,2025,0000138065,0000138065,2025,20250526,20250526,0000622491
6,0004056101,2025,0000138698,0000138698,2025,20250706,20250706,0000623060
7,0004056101,2025,0000139499,0000139499,2025,,,
8,0004056101,2025,0000139839,0000139839,2025,,,
9,0004055874,2024,0000132473,0000132473,2024,20241117,20241117,0000511095
